In [4]:
import boto3
import pandas as pd
import os
import ast

def handler(context, event):
    context.logger.info("Reading CSV")
    df = pd.read_csv(str(context.inputs['s3_images_csv']))
    
    if ast.literal_eval(str(context.inputs['download_data'])):
        context.logger.info("Initializing S3")
        s3 = boto3.resource('s3')
        bucket = s3.Bucket(str(context.inputs['bucket_name']))
        data_download_path = str(context.inputs['data_download_path'])

        context.logger.info("Creating Data Directories")
        os.makedirs(f"{data_download_path}/dog", exist_ok=True)
        os.makedirs(f"{data_download_path}/cat", exist_ok=True)

        context.logger.info("Downloading Dataset")
        for i, row in df.iterrows():
            local_image = f"{data_download_path}/{row['s3_image'].split('dogs_vs_cats/')[1]}"
            context.logger.debug(f"Downloading Image {i+1}/{len(df)}")
            bucket.download_file(row['s3_image'], local_image)
            df.at[i,'local_image'] = local_image
    
    context.logger.info("Logging Dataset / Path")
    context.log_dataset('s3_image_csv_local', df=df, format="csv", index=False)
    context.log_result("data_download_path", str(context.inputs['data_download_path']))